In [1]:
from deltalake import DeltaTable
import numpy as np
import pandas as pd

bets = DeltaTable("/Users/andreikirpichev/Downloads/bets/")
bets = bets.to_pandas()
bets.head()
bets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4871648 entries, 0 to 4871647
Data columns (total 23 columns):
 #   Column                Dtype         
---  ------                -----         
 0   customer_id           object        
 1   age                   int64         
 2   country               object        
 3   section               object        
 4   point                 object        
 5   market                object        
 6   market_params         object        
 7   contest_id            object        
 8   placement_date        object        
 9   placement_time        datetime64[ns]
 10  placement_odds        float32       
 11  home                  object        
 12  away                  object        
 13  region                object        
 14  sport                 object        
 15  league                object        
 16  minutes_before_start  float64       
 17  stake_eur             float64       
 18  dayofweek             int32         
 19  

In [2]:
from sklearn.preprocessing import LabelEncoder

categorical_features = ['country', 'section', 'point', 'market', 'market_params', 'contest_id', 'home', 'away', 'region', 'sport', 'league']

le = LabelEncoder()

# Convert the variables to numerical
for i in range(len(categorical_features)):
    label = le.fit_transform(bets[categorical_features[i]])
    bets[categorical_features[i]] = label
bets.head()


,customer_id,age,country,section,point,market,market_params,contest_id,placement_date,placement_time,...,region,sport,league,minutes_before_start,stake_eur,dayofweek,weekday_stake,section_2w,point_2w,market_2w
0,9786810,26,2,37,7,12,2320,15531,2022-09-01,2022-09-01 00:00:07.176,...,61,13,618,60.0,11.25,5,2.98,6.25,6.25,6.56
1,9187810,26,2,39,6,7,2320,63387,2022-09-01,2022-09-01 00:00:20.216,...,130,28,1484,0.0,5.00,5,3.56,3.02,3.25,2.94
2,9187810,26,2,86,6,9,2320,63380,2022-09-01,2022-09-01 00:00:20.216,...,130,28,1481,0.0,5.00,5,3.58,5.00,3.25,3.26
3,12072510,44,2,10,7,12,2320,15484,2022-09-01,2022-09-01 00:00:22.536,...,130,13,919,0.0,3.75,5,3.29,3.75,3.75,3.75
4,12072510,44,2,37,7,12,2320,15476,2022-09-01,2022-09-01 00:00:22.536,...,132,13,415,0.0,3.75,5,3.26,3.75,3.75,3.75


In [3]:
# Create features and target
X = bets[['customer_id', 'age', 'country', 'section', 'point', 'market', 'market_params', 'contest_id', 'home', 'away', 'region', 'sport', 'league', 'minutes_before_start', 'weekday_stake', 'section_2w', 'point_2w', 'market_2w']]
y = bets[['stake_eur']]

# show rows with NaN values
# X[X.isna().any(axis=1)]

# Make necessary imports
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 101)

# Train the model
regr = RandomForestRegressor(n_estimators = 10, max_depth = 10, random_state = 101)
regr.fit(X_train, y_train.values.ravel())

RandomForestRegressor(max_depth=10, n_estimators=10, random_state=101)

In [4]:
import warnings
warnings.filterwarnings('ignore')

# Make prediction
predictions = regr.predict(X_test)

result = X_test
result['stake_eur'] = y_test
result['prediction'] = predictions.tolist()
result.head()

,customer_id,age,country,section,point,market,market_params,contest_id,home,away,region,sport,league,minutes_before_start,weekday_stake,section_2w,point_2w,market_2w,stake_eur,prediction
1912175,3581510,31,2,37,7,4,2320,25938,13261,2429,43,13,1471,88.0,0.85,0.88,0.83,0.64,0.20,0.570315
2679583,16695110,27,2,37,7,0,2320,27296,16909,2878,43,13,1465,477.0,2.14,1.97,1.96,0.97,1.28,1.006688
2330869,16879510,57,2,37,7,4,2320,30134,19423,16475,107,13,1173,224.0,7.18,5.17,5.17,4.40,6.25,4.969072
2262266,8062010,51,2,37,7,4,2320,27740,19289,16545,33,13,45,130.0,0.41,0.44,0.44,0.41,0.23,0.384174
3504300,8592510,35,2,37,7,12,2320,48355,8695,9949,85,13,588,0.0,6.08,4.05,5.31,5.28,5.00,6.316569


In [5]:
# Import library for metrics
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Mean absolute error (MAE)
mae = mean_absolute_error(y_test.values.ravel(), predictions)

# Mean squared error (MSE)
mse = mean_squared_error(y_test.values.ravel(), predictions)

# R-squared scores
r2 = r2_score(y_test.values.ravel(), predictions)

# Print metrics
print('Mean Absolute Error:', round(mae, 2))
print('Mean Squared Error:', round(mse, 2))
print('R-squared scores:', round(r2, 2))

Mean Absolute Error: 5.17
Mean Squared Error: 801.37
R-squared scores: 0.63
